# Pytorch Data 的加载和表示  
## Time Series Data
主要目的是把原来的二维的按时间进行记录的数据表格变成是三维的

In [7]:
import numpy as np
import torch
import csv

In [3]:
bikes_numpy = np.loadtxt("./bike-sharing-dataset/hour-fixed.csv",
                        dtype=np.float32,
                        delimiter=',',
                        skiprows=1,
                        converters={1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

`np.loadtxt`函数  
- delimiter：分隔符
- skiprows：跳过行数，如果是1，那就是跳过首行，2-跳过两行
- converters：数据的预处理，不过这里是怎么处理的呢？ 
    - 首先1表示第一列，然后float函数用于数据的类型转换，lambda函数就是对这一列的数据进行了处理

C 代表通道，和一维数据的列数相同  
N 代表时间轴，每小时

In [8]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

现在要做的是把形态重塑一下，这个17520个小时的数据，如果能按照1天24小时再分一下会更好，那么就要分成三个维度

**`view`函数**  
可以改变查看storage中相同数据的方式，所以看到的形式不一样了  
view函数改变了数据的维度，同时不改变存储，零消耗  

In [9]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [10]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

**注意**  
下面这一部分实际上是希望把关于天气的评价情况变成一个one-hot向量的过程

In [11]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:, 9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [13]:
weather_onehot.scatter_(dim=1,
                       index=first_day[:, 9].unsqueeze(1)-1,  # -1的作用是把基数变成0开始，就是说把矩阵中元素都减1
                       value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [17]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

cat 函数  
torch.cat是将两个张量（tensor）拼接在一起，cat是concatnate的意思，即拼接，联系在一起

In [18]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [21]:
daily_weather_onehot.scatter_(1, 
                             daily_bikes[:, 9, :].long().unsqueeze(1)-1,
                             1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [23]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0    # turn the value between 0.0 and 1.0

上面是一种把数据归一化到0-1之间的方法，但是方法还有很多。  
比如最大值和最小值的利用，平均值和标准差等等